<a href="https://colab.research.google.com/github/nathanbollig/vet-graduate-expectations-survey/blob/main/SVM_WVMA_specialists_nontechnical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Veterinary graduate expectations survey

Evaluating the differences in graduate expectations between SVM specialists and WVMA specialists. Start by uploading the data into the working directory. This notebook focuses on the nontechnical questions, which are:
 -   Communication practices (Q13)
 -   Professional and business practices (Q14)
 -   Ethics and professional practices (Q16)
 
Two files are required:

1.   `SVM.xlsx`: SVM graduate expectations survey results
2.   `WVMA.xlsx`: WVMA graduate expectations survey results

In [1]:
! pip install xlsxwriter

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal

## Read in SVM data

In [3]:
# Use top row as header and skip second header row
svm = pd.read_excel('SVM.xlsx', header=0, skiprows=lambda x: x in [1])  

In [4]:
#svm.head(3)

In [5]:
# Read in questions from second header row and associate with column names
question_svm = {}

top_rows = pd.read_excel('SVM.xlsx', nrows=2) 

for col in list(top_rows.columns):
    question_svm[col] = top_rows.iloc[0][col]

## Read in WVMA data

In [6]:
# Use top row as header and skip second header row
wvma = pd.read_excel('WVMA.xlsx', header=0, skiprows=lambda x: x in [1])  

# Read in questions from second header row and associate with column names
question_wvma = {}

top_rows_wvma = pd.read_excel('WVMA.xlsx', nrows=2) 

for col in list(top_rows_wvma.columns):
    question_wvma[col] = top_rows_wvma.iloc[0][col]

In [7]:
def encode_expectation(response_string):
    if isinstance(response_string, int) == True:
        return response_string
    
    # Encode nan values as -1
    if isinstance(response_string, str) == False:
        if np.isnan(response_string) == True:
            return -1
    
    # Encode string
    s = response_string.lower()
    if s.find('no expectation') > -1:
        return 0
    elif s.find('with assistance') > -1:
        return 1
    elif s.find('indirect supervision') > -1:
        return 3
    elif s.find('direct supervision') > -1:
        return 2
    elif s.find('independently') > -1:
        return 4
    else:
        print(response_string)
        raise ValueError('Expected performance response was not formatted as expected.')

Generalists and specialists are defined by the answer to `wvma.Q49` which is equivalent to `svm.Q59`. We will assume that an empty answer corresponds to a generalist, and a non-empty answer is a specialist.

In [8]:
from collections import Counter
Counter(wvma.Q49)

Counter({'American Board of Veterinary Practitioners': 3,
         'American College of Theriogenologists': 2,
         'American College of Veterinary Anesthesia and Analgesia': 1,
         'American College of Veterinary Anesthesia and Analgesia,American College of Veterinary Dermatology,American College of Veterinary Emergency & Critical Care,American College of Veterinary Internal Medicine,American College of Veterinary Ophthalmologists,American College of Veterinary Surgeons,American Veterinary Dental College': 1,
         'American College of Veterinary Internal Medicine': 2,
         'American College of Veterinary Nutrition': 1,
         'American College of Veterinary Pathologists': 1,
         'American College of Veterinary Surgeons': 1,
         'American Veterinary Dental College': 1,
         'Other non-AVMA recognized specialty credentials': 9,
         nan: 153})

In [9]:
Counter(svm.Q59)

Counter({'American Board of Veterinary Practitioners': 2,
         'American College of Veterinary Anesthesia and Analgesia': 2,
         'American College of Veterinary Clinical Pharmacology,American College of Veterinary Internal Medicine': 1,
         'American College of Veterinary Dermatology': 1,
         'American College of Veterinary Emergency & Critical Care': 2,
         'American College of Veterinary Internal Medicine': 7,
         'American College of Veterinary Internal Medicine,American College of Veterinary Radiology': 1,
         'American College of Veterinary Nutrition': 1,
         'American College of Veterinary Ophthalmologists': 2,
         'American College of Veterinary Pathologists': 4,
         'American College of Veterinary Preventive Medicine,American College of Veterinary Radiology': 1,
         'American College of Veterinary Radiology': 3,
         'American College of Veterinary Surgeons': 5,
         'American College of Zoological Medicine': 3,
    

In [10]:
# Only include specialists
wvma = wvma[wvma['Q49'].notnull()].copy()
svm = svm[svm['Q59'].notnull()].copy()

In [11]:
len(svm)

38

In [12]:
len(wvma)

22

In [13]:
# Filter dataframe to only companion animal respondants (may have responded to other species too)
ca_svm = svm[svm['Q1'].str.contains('Companion Animal (canine and/or feline)', na=False, regex=False)].copy()

In [14]:
# Filter dataframe to only companion animal respondants (may have responded to other species too)
ca_wvma = wvma[wvma['Q1'].str.contains('Companion Animal (canine and/or feline)', na=False, regex=False)].copy()

## Question analysis code

In [15]:
def analyze_question(question_number, filtered_svm_df, filtered_wvma_df, n_subquestions, alpha=0.05, verbose=True):
    """
    Perform an analysis of a given question on a species-filtered dataframe.
    
    Inputs:
        question_number: main question number to analyze
        filtered_svm_df: svm dataframe filtered to respondants with the desired species area
        filtered_wvma_df: wvma dataframe filtered to respondants with the desired species area
        n_subquestions: number of subquestions in the main question
        alpha: power level for the statistical test

    Prints a summary of results.

    Outputs:
        table: summary table
        (pooled_stat, pooled_p, pooled_diff_mean): tuple of statistics describing output of Kruskal test on data pooled across subquestions
        svm_data: list of pooled svm data
        wvma_data: list of pooled wvma data
        sig_count: number of subquestions with significant difference detected (between svm and wvma responses), according to Kruskal test applied at subquestion level

    """

    svm_counts = np.zeros((n_subquestions, 6), dtype=int) # Row for each question, column for empty (-1), 0, 1, 2, 3, and 4 responses
    wvma_counts = np.zeros((n_subquestions, 6), dtype=int) # Row for each question, column for empty (-1), 0, 1, 2, 3, and 4 responses
    rows = []
    svm_pooled = []
    wvma_pooled = []
    sig_count = 0

    for i in range(1, n_subquestions+1):
        qkey = "Q" + str(question_number) + "_" + str(i)
        qstring = question_wvma[qkey].split('-')[2]

        # Encoding
        filtered_svm_df[qkey] = filtered_svm_df[qkey].apply(lambda x: encode_expectation(x))
        filtered_wvma_df[qkey] = filtered_wvma_df[qkey].apply(lambda x: encode_expectation(x))

        # svm tally
        counts = filtered_svm_df[qkey].value_counts(dropna=False)
        for key in counts.keys():
            svm_counts[i-1][key+1] += counts[key] # question index is 1-based; keys range from -1 to 4
        counts = svm_counts[i-1][1:] # counts of 0, 1, 2, 3, and 4
        svm_num_responses = np.sum(counts)
        svm_mean = (0*counts[0] + 1*counts[1] + 2*counts[2] + 3*counts[3] + 4*counts[4]) / svm_num_responses

        # wvma tally
        counts = filtered_wvma_df[qkey].value_counts(dropna=False)
        for key in counts.keys():
            wvma_counts[i-1][key+1] += counts[key]
        counts = wvma_counts[i-1][1:] # counts of 0, 1, 2, 3, and 4
        wvma_num_responses = np.sum(counts)
        wvma_mean = (0*counts[0] + 1*counts[1] + 2*counts[2] + 3*counts[3] + 4*counts[4]) / wvma_num_responses
        
        # Get data
        svm_data = list(filtered_svm_df[qkey])
        wvma_data = list(filtered_wvma_df[qkey])

        # Remove empty values from data
        svm_data = [x for x in svm_data if x != -1]
        wvma_data = [x for x in wvma_data if x != -1]

        assert(svm_num_responses == len(svm_data))
        assert(wvma_num_responses == len(wvma_data))

        # compare samples
        if len(svm_data) >= 5 and len(wvma_data) >= 5:
            stat, p = kruskal(svm_data, wvma_data)
        else:
            stat = 0
            p = 1

        # Determine significance
        if p > alpha:
            sig = ""
        else:
            sig = "*"
            sig_count += 1

        # Cache for pooled data
        svm_pooled.extend(svm_data)
        wvma_pooled.extend(wvma_data)

        # Cache for table of results
        row = [qstring] + list(svm_counts[i-1]) + [svm_mean, svm_num_responses] + list(wvma_counts[i-1]) + [wvma_mean, wvma_num_responses, svm_mean-wvma_mean, stat, p, sig]
        rows.append(row)

    # Assemble table of results
    table = pd.DataFrame(rows, columns=["Subquestion", "svm: empty", "svm: 0", "svm: 1", "svm: 2", "svm: 3", "svm: 4", "svm: avg", "svm: num responses", "wvma: empty", "wvma: 0", "wvma: 1", "wvma: 2", "wvma: 3", "wvma: 4", "wvma: avg", "wvma: num responses", "Diff Mean (svm-wvma)", "stat", "pval", "sig"])

    # Apply Kruskal test to pooled data
    pooled_stat, pooled_p = kruskal(svm_pooled, wvma_pooled)
    pooled_diff_mean = np.mean(svm_pooled) - np.mean(wvma_pooled)

    # Print
    if verbose == True:
        print('Pooled Q%s: stat=%.3f, p=%.2e, diff_mean (svm-wvma)=%.3f, sig_subq=%s/%s' % (question_number, pooled_stat, pooled_p, pooled_diff_mean, sig_count, n_subquestions))

    return table, (pooled_stat, pooled_p, pooled_diff_mean), svm_pooled, wvma_pooled, sig_count

In [16]:
table, subq_pooled_result, svm_data, wvma_data, sig_count = analyze_question(13, ca_svm, ca_wvma, n_subquestions=11)

Pooled Q13: stat=81.260, p=1.98e-19, diff_mean (svm-wvma)=0.808, sig_subq=10/11


In [17]:
table

,Subquestion,svm: empty,svm: 0,svm: 1,svm: 2,svm: 3,svm: 4,svm: avg,svm: num responses,wvma: empty,wvma: 0,wvma: 1,wvma: 2,wvma: 3,wvma: 4,wvma: avg,wvma: num responses,Diff Mean (svm-wvma),stat,pval,sig
0,Documents medical records to fulfill professi...,6,0,0,0,1,26,3.962963,27,5,0,1,0,4,5,3.300000,10,0.662963,11.331250,0.000762,*
1,Discuss recommended vaccination or preventive...,6,0,0,0,1,26,3.962963,27,5,2,1,1,1,5,2.600000,10,1.362963,11.815717,0.000587,*
2,Discuss recommended treatment plan and option...,6,0,0,0,2,25,3.925926,27,5,1,0,1,3,5,3.100000,10,0.825926,8.908561,0.002838,*
3,Discuss risks of recommended treatments and p...,7,0,0,0,3,23,3.884615,26,5,1,0,1,4,4,3.000000,10,0.884615,9.513225,0.002040,*
4,Acknowledge client’s knowledge level and appr...,6,0,0,0,1,26,3.962963,27,6,0,0,1,4,4,3.333333,9,0.629630,12.846675,0.000338,*
5,Discuss quality of life issues with owners,6,0,0,0,2,25,3.925926,27,5,1,0,0,4,5,3.200000,10,0.725926,8.644700,0.003280,*
6,Engages clients in difficult conversations su...,6,0,0,1,7,19,3.666667,27,5,1,2,0,5,2,2.500000,10,1.166667,8.966813,0.002749,*
7,Engage with clients and co,6,0,0,0,1,26,3.962963,27,5,0,0,0,0,10,4.000000,10,-0.037037,0.370370,0.542802,
8,Engage co,7,0,0,1,9,16,3.576923,26,5,1,2,1,3,3,2.500000,10,1.076923,5.319918,0.021083,*
9,"Elicit client goals, expectations, perspectiv...",6,0,0,0,2,25,3.925926,27,5,1,0,1,4,4,3.000000,10,0.925926,12.112294,0.000501,*


## Group analysis code

In [18]:
# cache data across all groups
group_data = []
group_columns = ["Group", "Pooled stat", "Pooled p", "Pooled diff_mean (svm-wvma)", "Num questions", "Fraction of sig questions", "Pooled num svm reponses", "Pooled num wvma responses"]

In [19]:
# cache tables
output_tables = []
output_tables_sheet_names = []

# cache subquestion table data
output_subq_data = []

In [20]:
# Input info about question group

question_list = [13,14,15]
n_subq_list = [11,6,8]
question_strings = ['Communication practices',
                    'Professional and business practices',
                    'Ethics and professional practices']

assert(len(question_list) == len(n_subq_list))
assert(len(n_subq_list) == len(question_strings))

In [21]:
# Code to analyze all questions within the group

def analyze_group(question_list, n_subq_list, question_strings, filtered_svm_df, filtered_wvma_df, alpha=0.05):
    svm_pooled = [] # now pooling over entire group
    wvma_pooled = []
    rows = []
    sig_count = 0
    subq_tables = []
    subq_tables_names = []

    for i in range(len(question_list)):
        question_number = question_list[i]
        n_subquestions = n_subq_list[i]
        question_string = question_strings[i]

        # Run analysis
        table, subq_pooled_result, svm_data, wvma_data, sig_subq = analyze_question(question_number, filtered_svm_df, filtered_wvma_df, n_subquestions, verbose=False)
        pooled_stat, pooled_p, pooled_diff_mean = subq_pooled_result
        svm_num_responses = len(svm_data)
        wvma_num_responses = len(wvma_data)

        # Cache procedure tables
        subq_tables.append(table)
        subq_tables_names.append('Q'+str(question_number))

        # Pool
        svm_pooled.extend(svm_data)
        wvma_pooled.extend(wvma_data)

        # Determine significance
        if pooled_p > alpha:
            sig = ""
        else:
            sig = "*"
            sig_count += 1

        # Cache data for group summary
        row = ['Q'+str(question_number), question_string, pooled_stat, pooled_p, sig, pooled_diff_mean, n_subquestions, "%i/%i" % (sig_subq,n_subquestions), svm_num_responses, wvma_num_responses]
        rows.append(row)

    # Assemble table of results
    group_table = pd.DataFrame(rows, columns=["Question number", "Category", "Pooled stat", "Pooled p", "Sig", "Pooled Diff Mean (svm-wvma)", "Num subquestions", "Fraction of sig subquestions", "Pooled num svm responses", "Pooled num wvma responses"])                     

    # Apply Kruskal test to pooled data
    pooled_stat, pooled_p = kruskal(svm_pooled, wvma_pooled)
    pooled_diff_mean = np.mean(svm_pooled) - np.mean(wvma_pooled)

    # Print
    print('Group result (all questions): stat=%.3f, p=%.2e, diff_mean (svm-wvma)=%.3f, sig_subq=%s/%s' % (pooled_stat, pooled_p, pooled_diff_mean, sig_count, len(question_list)))

    return group_table, (pooled_stat, pooled_p, pooled_diff_mean), sig_count, len(question_list), len(svm_pooled), len(wvma_pooled), (subq_tables, subq_tables_names)

In [22]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, ca_svm, ca_wvma)
pooled_stat, pooled_p, pooled_diff_mean = pooled_q_stats
group_data.append(["Companion Animal", pooled_stat, pooled_p, pooled_diff_mean, n_questions, "%i/%i" % (sig_count,n_questions), svm_responses, wvma_responses])
output_tables.append(group_table)
output_tables_sheet_names.append("Companion Animal")
output_subq_data.append(subq_data)
group_table

Group result (all questions): stat=99.923, p=1.58e-23, diff_mean (svm-wvma)=0.767, sig_subq=3/3


,Question number,Category,Pooled stat,Pooled p,Sig,Pooled Diff Mean (svm-wvma),Num subquestions,Fraction of sig subquestions,Pooled num svm responses,Pooled num wvma responses
0,Q13,Communication practices,81.260169,1.978748e-19,*,0.807961,11,10/11,295,109
1,Q14,Professional and business practices,30.493567,3.349754e-08,*,1.144444,6,4/6,162,60
2,Q15,Ethics and professional practices,11.860173,5.734721e-04,*,0.426389,8,2/8,216,80


In [23]:
# Filter dataframes to only companion animal respondants (may have responded to other species too)
ss_svm = svm[svm['Q1'].str.contains('Special Species', na=False, regex=False)].copy()
ss_wvma = wvma[wvma['Q1'].str.contains('Special Species', na=False, regex=False)].copy()

In [24]:
# Input info about question group

question_list = [13,14,15]
n_subq_list = [11,6,8]
question_strings = ['Communication practices',
                    'Professional and business practices',
                    'Ethics and professional practices']

assert(len(question_list) == len(n_subq_list))
assert(len(n_subq_list) == len(question_strings))

In [25]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, ss_svm, ss_wvma)
pooled_stat, pooled_p, pooled_diff_mean = pooled_q_stats
group_data.append(["Special Species", pooled_stat, pooled_p, pooled_diff_mean, n_questions, "%i/%i" % (sig_count,n_questions), svm_responses, wvma_responses])
output_tables.append(group_table)
output_tables_sheet_names.append("Special Species")
output_subq_data.append(subq_data)
group_table

Group result (all questions): stat=76.399, p=2.32e-18, diff_mean (svm-wvma)=2.236, sig_subq=3/3


,Question number,Category,Pooled stat,Pooled p,Sig,Pooled Diff Mean (svm-wvma),Num subquestions,Fraction of sig subquestions,Pooled num svm responses,Pooled num wvma responses
0,Q13,Communication practices,62.452974,2.728835e-15,*,2.666667,11,0/11,99,11
1,Q14,Professional and business practices,18.060708,2.139718e-05,*,3.388889,6,0/6,54,6
2,Q15,Ethics and professional practices,12.446267,4.188280e-04,*,0.777778,8,0/8,72,8


In [26]:
# Filter dataframes to only companion animal respondants (may have responded to other species too)
fa_svm = svm[svm['Q1'].str.contains('Food Animal', na=False, regex=False)].copy()
fa_wvma = wvma[wvma['Q1'].str.contains('Food Animal', na=False, regex=False)].copy()

In [27]:
# Input info about question group

question_list = [13,14,15]
n_subq_list = [11,6,8]
question_strings = ['Communication practices',
                    'Professional and business practices',
                    'Ethics and professional practices']


assert(len(question_list) == len(n_subq_list))
assert(len(n_subq_list) == len(question_strings))

In [28]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, fa_svm, fa_wvma)
pooled_stat, pooled_p, pooled_diff_mean = pooled_q_stats
group_data.append(["Food Animal", pooled_stat, pooled_p, pooled_diff_mean, n_questions, "%i/%i" % (sig_count,n_questions), svm_responses, wvma_responses])
output_tables.append(group_table)
output_tables_sheet_names.append("Food Animal")
output_subq_data.append(subq_data)
group_table

Group result (all questions): stat=19.610, p=9.50e-06, diff_mean (svm-wvma)=0.612, sig_subq=3/3


,Question number,Category,Pooled stat,Pooled p,Sig,Pooled Diff Mean (svm-wvma),Num subquestions,Fraction of sig subquestions,Pooled num svm responses,Pooled num wvma responses
0,Q13,Communication practices,8.902786,0.002847,*,0.627273,11,0/11,110,44
1,Q14,Professional and business practices,6.210323,0.012701,*,0.808333,6,0/6,60,24
2,Q15,Ethics and professional practices,5.603690,0.017923,*,0.443750,8,0/8,80,32


In [29]:
# Filter dataframes to only companion animal respondants (may have responded to other species too)
eq_svm = svm[svm['Q1'].str.contains('Equine', na=False, regex=False)].copy()
eq_wvma = wvma[wvma['Q1'].str.contains('Equine', na=False, regex=False)].copy()

In [30]:
# Input info about question group

question_list = [13,14,15]
n_subq_list = [11,6,8]
question_strings = ['Communication practices',
                    'Professional and business practices',
                    'Ethics and professional practices']


assert(len(question_list) == len(n_subq_list))
assert(len(n_subq_list) == len(question_strings))

In [31]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, eq_svm, eq_wvma)
pooled_stat, pooled_p, pooled_diff_mean = pooled_q_stats
group_data.append(["Equine", pooled_stat, pooled_p, pooled_diff_mean, n_questions, "%i/%i" % (sig_count,n_questions), svm_responses, wvma_responses])
output_tables.append(group_table)
output_tables_sheet_names.append("Equine")
output_subq_data.append(subq_data)
group_table

Group result (all questions): stat=35.056, p=3.20e-09, diff_mean (svm-wvma)=0.960, sig_subq=3/3


,Question number,Category,Pooled stat,Pooled p,Sig,Pooled Diff Mean (svm-wvma),Num subquestions,Fraction of sig subquestions,Pooled num svm responses,Pooled num wvma responses
0,Q13,Communication practices,20.618330,0.000006,*,0.965909,11,0/11,88,33
1,Q14,Professional and business practices,8.749698,0.003097,*,1.430556,6,0/6,48,18
2,Q15,Ethics and professional practices,9.249225,0.002356,*,0.598958,8,0/8,64,24


## Group Summary

In [32]:
group_summary_table = pd.DataFrame(group_data, columns=group_columns)

In [33]:
ALPHA = 0.05

pvals = list(group_summary_table['Pooled p'])

sigs = []
for p in pvals:
  if p > ALPHA:
      sig = ""
  else:
      sig = "*"
  sigs.append(sig)

group_summary_table.insert(loc=3, column='Sig', value=sigs)

In [34]:
# Add group summary to the beginning of output tables
output_tables.insert(0, group_summary_table)
output_tables_sheet_names.insert(0, "Group summary")

In [35]:
group_summary_table

,Group,Pooled stat,Pooled p,Sig,Pooled diff_mean (svm-wvma),Num questions,Fraction of sig questions,Pooled num svm reponses,Pooled num wvma responses
0,Companion Animal,99.923031,1.584361e-23,*,0.766722,3,3/3,673,249
1,Special Species,76.398699,2.318040e-18,*,2.235556,3,3/3,225,25
2,Food Animal,19.610397,9.495108e-06,*,0.612000,3,3/3,250,100
3,Equine,35.056463,3.202821e-09,*,0.960000,3,3/3,200,75


# Generate tables

We will generate the following tables using pooled data from these experiments:

1.   `summary_s_nontechnical.xlsx`: Group summary table and a table for procedure sets (questions) within each group.
2.   `companion_animal_s_nontechnical.xlsx`: Tables for all procedures within the companion animal group.
3.   `special_species_s_nontechnical.xlsx`: Tables for all procedures within the special species group.
4.   `food_animal_s_nontechnical.xlsx`: Tables for all procedures within the food animal group.
5.   `equine_s_nontechnical.xlsx`:Tables for all procedures within the equine group.
6. `summary_s_nontechnical_allspecies.xlsx`: Summary table for responses to procedures (subquestions) pooled across species areas.



## Summary

In [36]:
writer = pd.ExcelWriter('summary_s_nontechnical.xlsx', engine='xlsxwriter')

for i,table in enumerate(output_tables):
    sheet_name = output_tables_sheet_names[i]
    table.to_excel(writer, sheet_name=sheet_name, index=False)

    # Auto-adjust columns widths
    for column in table:
        column_width = max(table[column].astype(str).map(len).max(), len(column))
        col_idx = table.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

writer.save()

## All Species Summary

In [37]:
group_table, pooled_q_stats, sig_count, n_questions, svm_responses, wvma_responses, subq_data  = analyze_group(question_list, n_subq_list, question_strings, svm, wvma)

Group result (all questions): stat=84.579, p=3.69e-20, diff_mean (svm-wvma)=0.568, sig_subq=3/3


In [38]:
subq_tables, subq_tables_names = subq_data

# Loop through tables
writer = pd.ExcelWriter('summary_s_nontechnical_allspecies.xlsx', engine='xlsxwriter')

for i,table in enumerate(subq_tables):
    sheet_name = subq_tables_names[i]
    table.to_excel(writer, sheet_name=sheet_name, index=False)

    # Auto-adjust columns widths
    for column in table:
        column_width = max(table[column].astype(str).map(len).max(), len(column))
        col_idx = table.columns.get_loc(column)
        writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

writer.save()

## All procedures

In [39]:
for i, file in enumerate(['companion_animal_s_nontechnical.xlsx', 'special_species_s_nontechnical.xlsx', 'food_animal_s_nontechnical.xlsx', 'equine_s_nontechnical.xlsx']):
    subq_data = output_subq_data[i]
    subq_tables, subq_tables_names = subq_data

    # Loop through tables
    writer = pd.ExcelWriter(file, engine='xlsxwriter')

    for i,table in enumerate(subq_tables):
        sheet_name = subq_tables_names[i]
        table.to_excel(writer, sheet_name=sheet_name, index=False)

        # Auto-adjust columns widths
        for column in table:
            column_width = max(table[column].astype(str).map(len).max(), len(column))
            col_idx = table.columns.get_loc(column)
            writer.sheets[sheet_name].set_column(col_idx, col_idx, column_width)

    writer.save()